<a href="https://colab.research.google.com/github/mohamad1371/word-auto-correct-with-python/blob/main/word_auto_correct_with_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import re
from collections import Counter
import numpy as np
import pandas as pd

def delete_letter(word):
    
    delete_l = []
    word = word.lower()
    for index in range(len(word)):
        s = word[:index] + word[index + 1:]
        if s not in delete_l and len(s)>0:
            delete_l.append(s)
    return delete_l

def replace_letter(word):
    replace_l=[]
    word = word.lower()
    for i in range(len(word)):
        for j in range(ord('a'), ord('z')+1):
            w = word[:i] + chr(j) + word[i+1:]
            if w not in replace_l and w != word:
                replace_l.append(w)

    return replace_l
    
def insert_letter(word):
    insert_l = []
    word = word.lower()
    word_list = list(word)
    for i in range(len(word)+1):
        for j in range(ord('a'), ord('z')+1):
            word_list.insert(i,chr(j))
            if ''.join(word_list) not in insert_l:
                insert_l.append(''.join(word_list))
            del(word_list[i])

    return insert_l

def calculate_cost(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1
    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2    

    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 2

    return distances[len(token1)][len(token2)]    



def edit_one_letter(word):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    edit_one_set = set()
    edit_one_set.update(delete_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))
    return edit_one_set
    
def edit_two_letters(word):
    edit_two_set = set()
    all_1_changed = edit_one_letter(word)
    for i in all_1_changed:
        edit_two_set.update(delete_letter(i), replace_letter(i), insert_letter(i))

    return edit_two_set   


def autoCorrect(word):

    with open('shakespeare.txt') as f:
        data = f.read()

    data = re.sub('[^A-Za-z0-9]+', ' ', data)    
    all_refwords_list = re.split(r'\W+', data)
    all_refwords_set = set(all_refwords_list)
    one_change = edit_one_letter(word)
    two_change = edit_two_letters(word)
    all_1_2_change = one_change.union(two_change)
    usable_words  = list(all_1_2_change & all_refwords_set)
    repeat_num_dic = dict((i, all_refwords_list.count(i)) for i in usable_words)

    distance_dic = {}
    for i in repeat_num_dic.keys():
        distance_dic[i] = calculate_cost(word, i)
    distance_dic = {k: v for k, v in sorted(distance_dic.items(), key=lambda item: item[1])}
    special_words = [i for i in distance_dic.keys() if distance_dic[i]==distance_dic[list(distance_dic.keys())[0]]]
    
    if len(special_words) == 1:
        result = special_words[0]

    elif  len(special_words) == 0:
        print('There is no matching word with less than 4 penalties')
        result = None
    else:
        list_of_best_specils = [[repeat_num_dic[i], i] for i in special_words]
        list_of_best_specils = sorted(list_of_best_specils)
        result = list_of_best_specils[-1][1]



    return result

input_word = input('Inter a misspelled word: ')

autoCorrect(input_word)

Inter a misspelled word: natch


'snatch'